In [1]:
import geopandas as gpd
import os
from datetime import datetime
import satsearch
from scipy.ndimage import zoom
from tqdm import tqdm
import rasterio
import certifi
import boto3
from botocore.client import Config
import sys
from pyproj import Transformer
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def MNDWI_Images(gdf, main_path, mndwi_threshold=0):
    os.makedirs(main_path, exist_ok=True)
    
    all_water_extent_data = {}
    
    for id in gdf.Name:
        water_extent_data = {}
        id_path = os.path.join(main_path, id)
        os.makedirs(id_path, exist_ok=True)

        bbox = gdf[gdf["Name"] == id].reset_index(drop=True).total_bounds.tolist()
        start_date = datetime.strptime("2024-01-01", '%Y-%m-%d').strftime('%Y-%m-%dT00:00:00Z')
        end_date = datetime.strptime("2024-05-22", '%Y-%m-%d').strftime('%Y-%m-%dT23:59:59Z')
        timeRange = f'{start_date}/{end_date}'
        logging.info(f'Processing ID: {id} for time range: {timeRange}')
        
        SentinelSearch = satsearch.Search.search(
            url="https://earth-search.aws.element84.com/v1",
            bbox=bbox,
            datetime=timeRange,
            collections=['sentinel-2-l2a'],
            limit=1000
        )
        
        Sentinel_items = SentinelSearch.items()
        logging.info(Sentinel_items.summary(['date', 'id', 'eo:cloud_cover']))
        
        # Find the image with the least cloud cover for each unique date
        date_cloud_dict = {}
        
        for item in Sentinel_items:
            date = item.properties['datetime'][0:10]
            cloud_cover = item.properties['eo:cloud_cover']
            if cloud_cover > 10:
                continue
            if date not in date_cloud_dict or cloud_cover < date_cloud_dict[date]['cloud_cover']:
                date_cloud_dict[date] = {
                    'cloud_cover': cloud_cover,
                    'item': item
                }
        
        for date, value in tqdm(date_cloud_dict.items()):
            item = value['item']
            green_s3 = item.assets['green']['href']  # Green band
            swir1_s3 = item.assets['nir']['href']  # SWIR1 band
            scl_s3 = item.assets['scl']['href']    # Scene classification map
            
            try:
                scl, kwargs = getSubset(scl_s3, bbox)
                logging.info(f"SCL unique values: {np.unique(scl)}")  # Debugging SCL values
                
                green, kwargs = getSubset(green_s3, bbox)
                swir1, kwargs = getSubset(swir1_s3, bbox)
                
                if not validate_image(scl):
                    logging.info(f'Image on {date} is not valid due to cloud or other artifacts.')
                    continue

            except Exception as e:
                logging.error(f'Error processing field id {id}: {e}')
                continue

            mndwi = compute_MNDWI(green, swir1)
            
            # Compute water extent using the threshold value
            water_extent = np.sum(mndwi > mndwi_threshold)
            
            water_extent_data[date] = water_extent

            # Save the water extent images
            plt.imshow(mndwi, cmap='Blues', extent=bbox)
            plt.colorbar(label='MNDWI')
            plt.title(f'Water Extent on {date}')
            plt.savefig(f'{id_path}/{date}_{id}_water_extent.png')
            plt.close()
            
            del scl, mndwi, green, swir1
        
        # Save water extent data for this ID
        all_water_extent_data[id] = water_extent_data
        
        # Plot time series line chart for water extent for this ID
        dates = sorted(water_extent_data.keys())
        water_extents = [water_extent_data[date] for date in dates]
        print(water_extents)
        
        # Identify and handle outliers
        z_threshold = 2.5  # Z-score threshold for outlier detection
        z_scores = np.abs(stats.zscore(water_extents))
        
        non_outliers = np.where(z_scores <= z_threshold)[0]
        
        dates = [dates[i] for i in non_outliers]
        water_extents = [water_extents[i] for i in non_outliers]
            
        plt.figure(figsize=(10, 6))
        plt.plot(dates, water_extents, marker='o', linestyle='-')
        plt.xlabel('Date')
        plt.ylabel('Water Extent (in pixels)')
        plt.title(f'Water Extent Time Series for ID {id}')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(id_path, f'water_extent_time_series_{id}.png'))
        plt.close()

       
    # Return all water extent data
    return all_water_extent_data

def getSubset(geotiff_file, bbox):
    with rasterio.Env(aws_session):
        with rasterio.open(geotiff_file) as geo_fp:
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs) 
            lat_north, lon_west = Transf.transform(bbox[3], bbox[0])
            lat_south, lon_east = Transf.transform(bbox[1], bbox[2]) 
            x_top, y_top = geo_fp.index(lat_north, lon_west)
            x_bottom, y_bottom = geo_fp.index(lat_south, lon_east)
            window = rasterio.windows.Window.from_slices((x_top, x_bottom), (y_top, y_bottom))
            subset = geo_fp.read(1, window=window)
            kwargs = geo_fp.meta.copy()
            kwargs.update({
                'height': window.height,
                'width': window.width,
                'transform': rasterio.windows.transform(window, geo_fp.transform),
                'dtype': 'uint16'  # Update data type as per band data type
            })
    return subset, kwargs

def validate_image(scl):
    """
    Validate the image based on the scene classification layer (SCL).
    Returns True if the image is good, False otherwise.
    """
    # Define the valid classes for SCL (e.g., excluding clouds, shadows, etc.)
    valid_classes = [4, 5, 6, 7, 11]  # Example: vegetation, bare soil, water, unclassified, snow/ice
    
    invalid_pixels = np.isin(scl, valid_classes, invert=True).sum()
    total_pixels = scl.size
    invalid_ratio = invalid_pixels / total_pixels

    logging.info(f'Invalid pixel ratio: {invalid_ratio:.2%}')
    
    # Set a threshold for the acceptable ratio of invalid pixels
    invalid_threshold = 0.2  # 20% of invalid pixels
    return invalid_ratio <= invalid_threshold

def compute_MNDWI(green, swir1):
    # Resize the SWIR1 band array to match the dimensions of the green band array
    swir1_resized = swir1  # zoom(swir1, np.array(green.shape) / np.array(swir1.shape), order=1)
    
    # Compute MNDWI
    with np.errstate(divide='ignore', invalid='ignore'):
        mndwi = np.where((green + swir1_resized) != 0,
                         (green.astype(float) - swir1_resized.astype(float)) / (green + swir1_resized),
                         0)
    return mndwi

# Set environment variables for SSL certificates and AWS session
os.environ['REQUESTS_CA_BUNDLE'] = os.path.join(os.path.dirname(sys.argv[0]), 'cacert.pem')
logging.info(certifi.where())
os.environ['REQUESTS_CA_BUNDLE'] = os.path.join(os.path.dirname(sys.argv[0]), certifi.where())
os.environ['CURL_CA_BUNDLE'] = str(certifi.where())
aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)

# Read geojson file and process images
rdf = gpd.read_file("aoi.geojson")
rdf = rdf[['Name', 'geometry']]
all_water_extent_data = MNDWI_Images(rdf, "/home/purna/Desktop/Dvara/mlai/NatureDots/images", mndwi_threshold=0.1)

# Print or save the collected water extent data
#print(all_water_extent_data)


2024-05-24 18:19:02,470 - INFO - /home/purna/Desktop/Dvara/mlai/mlai_env/lib/python3.10/site-packages/certifi/cacert.pem
2024-05-24 18:19:04,570 - INFO - Processing ID: pond1 for time range: 2024-01-01T00:00:00Z/2024-05-22T23:59:59Z
2024-05-24 18:19:14,061 - INFO - Items (58):
date                      id                        eo:cloud_cover            
2024-05-21                S2B_43QCD_20240521_0_L2A  10.689957                 
2024-05-18                S2B_43QCD_20240518_0_L2A  0.02295                   
2024-05-16                S2A_43QCD_20240516_0_L2A  2.802207                  
2024-05-13                S2A_43QCD_20240513_0_L2A  13.74899                  
2024-05-11                S2B_43QCD_20240511_0_L2A  31.155202                 
2024-05-08                S2B_43QCD_20240508_0_L2A  0.027788                  
2024-05-06                S2A_43QCD_20240506_0_L2A  15.821746                 
2024-05-03                S2A_43QCD_20240503_0_L2A  0.002905                  
2024-05-01 

[19082, 27524, 28483, 28223, 26992, 27704, 28188, 28542, 28142, 28092, 28505, 28097, 27921, 27682, 27013, 27840, 27109, 26546, 25718, 26381, 26625, 27071, 16529, 26533, 26185, 24698, 24050, 24187]


2024-05-24 18:38:18,535 - INFO - Processing ID: pond2 for time range: 2024-01-01T00:00:00Z/2024-05-22T23:59:59Z
2024-05-24 18:38:26,795 - INFO - Items (27):
date                      id                        eo:cloud_cover            
2024-05-19                S2A_42RXQ_20240519_0_L2A  12.918945                 
2024-05-14                S2B_42RXQ_20240514_0_L2A  4.6e-05                   
2024-05-09                S2A_42RXQ_20240509_0_L2A  0                         
2024-05-04                S2B_42RXQ_20240504_0_L2A  40.556961                 
2024-04-29                S2A_42RXQ_20240429_0_L2A  4e-05                     
2024-04-24                S2B_42RXQ_20240424_0_L2A  5.051775                  
2024-04-19                S2A_42RXQ_20240419_0_L2A  0.007097                  
2024-04-14                S2B_42RXQ_20240414_0_L2A  99.90198                  
2024-04-09                S2A_42RXQ_20240409_0_L2A  74.454868                 
2024-04-04                S2B_42RXQ_20240404_0_L2A  7

[1274, 1210, 1154, 1127, 1110, 1079, 919, 903, 845, 796, 139, 754, 722]
{'pond1': {'2024-05-18': 24187, '2024-05-08': 24050, '2024-05-03': 24698, '2024-04-23': 26185, '2024-04-08': 26533, '2024-04-06': 16529, '2024-04-03': 27071, '2024-03-27': 26625, '2024-03-24': 26381, '2024-03-19': 25718, '2024-03-14': 26546, '2024-03-12': 27109, '2024-03-04': 27840, '2024-02-28': 27013, '2024-02-23': 27682, '2024-02-18': 27921, '2024-02-16': 28097, '2024-02-11': 28505, '2024-02-06': 28092, '2024-02-03': 28142, '2024-02-01': 28542, '2024-01-24': 28188, '2024-01-22': 27704, '2024-01-19': 26992, '2024-01-17': 28223, '2024-01-14': 28483, '2024-01-12': 27524, '2024-01-04': 19082}, 'pond2': {'2024-05-14': 722, '2024-05-09': 754, '2024-04-29': 139, '2024-04-24': 796, '2024-04-19': 845, '2024-03-30': 903, '2024-03-15': 919, '2024-02-24': 1079, '2024-02-19': 1110, '2024-02-14': 1127, '2024-02-09': 1154, '2024-01-20': 1210, '2024-01-10': 1274}}
